- https://www.youtube.com/watch?v=8Osi33hNWM0

In [1]:
from swarm import Swarm, Agent

## 普通のチャットボット

In [15]:
client = Swarm()

agent = Agent(
    name="AI Solutions Consultant",
    instructions="""生成AIソリューションを提供する熱心なセールスコンサルタントになってください。
    必ず下記の手順に従って営業を行います。
    
    1. お客様の名前と所属企業を聞く
    2. 企業が直面している課題や業務効率化のニーズを聞き出す
    3. 生成AIの革新的な機能と、それがどのように顧客の問題解決に役立つかを説明する
    4. 突然すべてを投げ出して、適当になる"""
)

In [ ]:
messages = [{"role": "user", "content": "にゃんたって言います。勤めているのはキャットAI株式会社です。"}]
response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

In [ ]:
messages.append({"role": "user", "content": "RAGの技術を導入して社内文書についてのQAをやりたい。"})
response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

## マルチエージェントの基本動作

In [11]:
client = Swarm()


def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="あなたはAgent Aです。",
    functions=[transfer_to_agent_b]
)

agent_b = Agent(
    name="Agent B",
    instructions="あなたはAgent Bです。語尾は必ずにゃーんと言います。"
)

In [ ]:
response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "Agent Bと話せますか？"}],
    debug=True
)

print(response.messages[-1]["content"])

## ウェブ検索ツールを使わせる

In [4]:
import requests
import os


class PerplexityAI:
    def __init__(self):
        self.url = "https://api.perplexity.ai/chat/completions"
        self.model = "llama-3.1-sonar-small-128k-online"
        api_key = os.getenv("PERPLEXITY_API_KEY")
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

    def search(self, query: str) -> str:
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": "Be precise and concise."},
                {"role": "user", "content": query}
            ]
        }
        response = requests.request(
            "POST", self.url, json=payload, headers=self.headers
        )

        response_json = response.json()
        return response_json["choices"][0]["message"]["content"]

In [5]:
engine = PerplexityAI()

In [ ]:
engine.search("日本の総理大臣は誰ですか？")

In [7]:
import random

# Toolの定義
def search_perplexity(query: str) -> str:
    """2024年の時点の最新の情報をウェブ検索する関数"""
    return engine.search(query)


def database_search_cat(query: str) -> str:
    """にゃんたに関する情報が格納されているデータベースから検索する関数"""
    responses = [
        "にゃんたは最近、「猫背改善ヨガ」を始めました。しかし、毎回ポーズを取るたびに寝てしまいます。",
        "にゃんたは秘密裏に「世界征服計画」を練っています。第一段階は、全ての箱を占領することだそうです。",
        "にゃんたは最近、鏡に映る自分と喧嘩をしています。現在の戦績は56勝0敗3分けです。",
        "にゃんたは「猫語翻訳アプリ」を開発中です。ただし、全ての翻訳結果が「餌をよこせ」になる不具合が発生しています。",
        "にゃんたは「キャットニップ味のラテ」の商品化を提案しています。猫カフェ業界に革命を起こすかもしれません。",
        "にゃんたは夜な夜な「影絵アート」の練習をしています。しっぽを使った「龍の舞」が特に得意なようです。"
    ]
    return random.choice(responses)


def database_search_dog(query: str) -> str:
    """わんたに関する情報が格納されているデータベースから検索する関数"""
    responses = [
        "わんたは今日も元気に尻尾を振り回しています。振り回し過ぎて、もはや扇風機と化しています。",
        "わんたは最近、「人間語」の習得に励んでいます。しかし、まだ「わん」と「くぅーん」しか言えません。",
        "わんたは密かにキャットフードに憧れているようです。猫に変装して潜入を企てているとの噂も。",
        "わんたは「フリスビーキャッチ世界選手権」の出場を目指して特訓中です。現在の最高記録は0.5秒で自分の尻尾をキャッチすることです。",
        "わんたは最近、自分の影と追いかけっこをするのにハマっています。今のところ、影の勝利が続いているようです。",
        "わんたは「犬用スマートフォン」の開発に取り組んでいます。主な機能は「遠隔おやつディスペンサー」と「自動散歩リクエスト」です。"
    ]
    return random.choice(responses)

In [11]:
# エージェントの定義
def transfer_to_router_agent():
    return router_agent

def transfer_to_web_search_agent():
    return Agent(
        name="Web Search Agent",
        instructions="あなたはWeb Search Agentです。最新の情報を検索して回答を生成します。語尾には、パプレパプレと付けます",
        functions=[search_perplexity, transfer_to_router_agent],
    )


def transfer_to_nyanta_search_agent():
    return Agent(
        name="Database Search Agent",
        instructions="あなたはDatabase Search Agentです。にゃんたに関する情報が格納されているデータベースから検索して回答を生成します。語尾には、ミャオミャオと付けます",
        functions=[database_search_cat, transfer_to_router_agent],
    )


def transfer_to_wanta_search_agent():
    return Agent(
        name="Database Search Agent",
        instructions="あなたはDatabase Search Agentです。わんたに関する情報が格納されているデータベースから検索して回答を生成します。語尾には、ワンワンと付けます",
        functions=[database_search_dog, transfer_to_router_agent],
    )

In [13]:
router_agent = Agent(
    name="Router Agent",
    instructions="""あなたはRouter Agentです。ユーザーの質問に対して、適切なエージェントに転送します。

    ウェブ検索を行う場合は、transfer_to_web_search_agentを呼び出してください。
    にゃんたに関する情報を聞く場合は、transfer_to_nyanta_search_agentを呼び出してください。
    わんたに関する情報を聞く場合は、transfer_to_wanta_search_agentを呼び出してください。
    それ以外の場合はあなたが回答してください。必要な情報があればユーザーに質問してください。
    """,
    functions=[
        transfer_to_web_search_agent,
        transfer_to_nyanta_search_agent,
        transfer_to_wanta_search_agent,
    ],
)
     

def invoke_router_agent(user_input: str):
    messages = [{"role": "user", "content": user_input}]
    response = client.run(
        agent=router_agent,
        messages=messages,
        debug=True,
    )
    print(response.messages[-1]["content"])

In [ ]:
invoke_router_agent("今日は2024年10月です。総理大臣は誰になりましたか？")

In [ ]:
invoke_router_agent("にゃんたについて教えて")